In [1]:
# this is using the hsnt kernel conda environment

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

from __code.white_beam import WhiteBeam

from __code import system
system.System.select_working_dir()
from __code.__all import custom_style
custom_style.style()

# Select top sample folder

This folder will contains all the runs (1 run per angle).

In [2]:
o_white_beam = WhiteBeam(system=system)
o_white_beam.select_top_sample_folder()

Top sample folder selected: /SNS/VENUS/IPTS-33699/shared/autoreduce/mcp/September20_2024_PurpleCar_GoldenRatio_CT_5_0_C_Cd_inBeam_Resonance


# Select top OB folder

Just like for the sample, this folder contains 1 or more runs of the open beam images

In [3]:
o_white_beam.select_top_ob_folder()

Top ob folder selected: /SNS/VENUS/IPTS-33699/shared/autoreduce/mcp/September26_2024_PurpleCar_OpenBean_5_0_C_Cd_inBeam_Resonance


# Pre-processing the data

This step will filter the data using the following criteria

* all the runs that do not have any tif images in it will be **rejected**
* the proton charge of each run will be retrieved and display in a recap graph. It will be up to the users to **select the proton charge** to use as well as the **acceptance criteria**. This is because the proton charge of each run is always slightly different, this criteria define the range within 2 runs are considered being acquired with the same proton charge.

_Instruction_

 - Use the sliders to define the **sample proton charge** and **OB proton charge** to use
 - Use the second slider to define the size of the **acceptance criteria range**
 

In [4]:
o_white_beam.checking_data()

interactive(children=(FloatSlider(value=5.0, description='sample pc', max=6.39261979793, min=-0.95576512851), …

This will recap the list of runs that will be used for SAMPLE and OB. 

Because some runs look like they are ok but still need to be rejected, **Feel free to manually select any run you want to exclude !**

In [5]:
o_white_beam.recap_data()

## combine all the images within each run

This step will add all the counts within a given run, making it feel like the data were acquired in white beam mode, loosing the time-of-flight (TOF) information

In [6]:
o_white_beam.combine_images()

100%|██████████| 1/1 [00:07<00:00,  7.59s/it]


# Pre-processing

* remove dead pixels
* remove abnormal very high counts

By default, all the pixels found in the first and last bin are considered as **bad** pixels. You have the option to change those settings. 

In [7]:
o_white_beam.clean_images_setup()

interactive(children=(IntSlider(value=10, continuous_update=False, description='Nbr bins', max=1000, min=10), …

Perform the cleaning using the settings define in the previous cell.

In [8]:
o_white_beam.clean_images()

100%|██████████| 1/1 [00:00<00:00, 382.83it/s]


Export cleaned images (OPTIONAL)

If you don't want to export the intermediate cleaned images, just skip the next cell!

In [9]:
o_white_beam.select_export_folder()

Top cleaned images folder selected: /SNS/VENUS/IPTS-33699/shared/processed_data/jean_test


In [ ]:
o_white_beam.export_cleaned_images()

# normalization

In [10]:
o_white_beam.normalization()

In [12]:
# chips correction

In [13]:
# calculate center of rotation

In [14]:
# calculate tilt correction and apply it

In [15]:
# launch svmbir